Reference:  
https://doc.codingdict.com/sklearn/28/ inverse covmatrix  
https://github.com/cran/GRS.test/blob/master/R/GRS.test.R code

In [372]:
import pandas as pd
import numpy as np 
import statsmodels.api as sm
from scipy.stats import t
from scipy.stats import f
factor_mat=pd.read_csv('factor_mat.csv').iloc[:,1:]
ret_mat=pd.read_csv('ret_mat.csv').iloc[:,1:]
grs=grs_test(ret_mat,factor_mat)
print(grs['stat'])

{'F': 1.6048790340443542, 'F_p': 0.03585358766354907}


In [371]:
def grs_test(ret_mat,factor_mat):
    r_mat=ret_mat.values
    f_mat=factor_mat.values
    N=ret_mat.shape[1] #portfolio numbers
    T=ret_mat.shape[0] #Time
    K = factor_mat.shape[1]  #model factor

    emat=np.zeros((T, N))
    bmat=np.zeros((N, K+1)) #include intercept
    semat=np.zeros((N, K+1))
    r2mat=np.zeros((N, 1))
    tmat=np.zeros((N, K+1))
    pmat=np.zeros((N, K+1))
    dat=sm.add_constant(f_mat)

    for i in range(0,N):
        ri=r_mat[:,i]
        b=(np.linalg.inv(dat.T@dat)@dat.T)@ri #beta coef
        e = ri -dat @b #residual
        bmat[i,:] = b 
        emat[:,i]=e
        r2mat[i] = 1-(np.sum(e**2)/np.sum((ri-np.mean(ri))**2))
        s2=np.sum(e**2)/(T-K-1) #mse
        covmat=np.linalg.inv(dat.T@dat)*s2 #precision matrix
        semat[i,:]=np.diag(covmat)**0.5
        tmat[i,:] = b/(np.diag(covmat)**0.5)
        pmat[i,:]  = 2*(1 - t.cdf(abs(b/np.diag(covmat)**0.5), T-K-1))


    sigma=(emat.T @emat )/(T-K-1) #error cov
    alpha=bmat[:,0]
    factor_mean=f_mat.mean(0)
    omega=((f_mat-factor_mean).T @(f_mat-factor_mean))/(T-1) #demean factor cov
    tem1=(alpha.T)@np.linalg.inv(sigma)@alpha
    tem2=(factor_mean @ np.linalg.inv(omega)@(factor_mean.T))+1
    tem3=T/N
    tem4=(T-N-K)/(T-K-1)
    F = tem3*tem4 *(tem1/tem2)
    pF = 1-f.cdf(F, dfn=N, dfd=T-N-K)
    if f_mat.shape[1] == 1:
        factornames = ["intercept","Singlefactor"] 
    else:
        factornames = ['intercept']+list(factor_mat.columns)
    retnames=list(ret_mat)

    tab=['stat','b','e','t','p','se','r']
    matlist=[bmat,emat,tmat,pmat,semat,r2mat]
    result_dict=dict()
    for i in tab:
        result_dict[i]=dict()

    for i in range(0,len(tab)):
        if tab[i]=='stat':
            result_dict[tab[i]]['F']=F
            result_dict[tab[i]]['F_p']=pF
        else:
            mat=matlist[i-1]
            if tab[i]!='r':
                 for j in range(0,N):
                    result_dict[tab[i]][retnames[j]]=mat[j,:]

            else:
                 for j in range(0,N):
                    result_dict[tab[i]][retnames[j]]=mat[j]
    return result_dict